In [17]:
import sys
import subprocess

# implement pip as a subprocess:
packages = ['pandas','numpy','pyarrow']
for p in packages:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install',p])


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [18]:
import os
import pandas as pd
import numpy as np
import io
import fnmatch
import tarfile
from ftplib import FTP,error_perm
from IPython.display import display

In [3]:
paths_to_df = [
 ('targets.feather','targets.tar.gz','pub/databases/opentargets/platform/21.11/output/etl/parquet/targets')
,('diseases.feather','diseases.tar.gz','pub/databases/opentargets/platform/21.11/output/etl/parquet/diseases')
,('evidence.feather','evidence.tar.gz','pub/databases/opentargets/platform/21.11/output/etl/parquet/evidence/sourceId=eva')
]

In [4]:
def load_datasets_with_fallback():
    try:
        for feather_path,parquet_path,ftp_path in paths_to_df:
            if os.path.isfile(feather_path):
                yield pd.read_feather(feather_path)
            elif os.path.isfile(parquet_path):
                with tarfile.open(parquet_path, "r:*") as tar:
                    df = pd.concat([pd.read_parquet(tar.extractfile(file)) for file in tar.getnames()])
                    df.reset_index().to_feather(feather_path,compression = 'lz4')
                    yield pd.read_feather(feather_path)
            else:
                ftp = FTP('ftp.ebi.ac.uk')
                ftp.login()
                ftp.cwd(path)
                files = [filename for filename in ftp.nlst() if fnmatch.fnmatch(filename, '*.parquet')]
                with tarfile.open(parquet_path, "w:gz") as fout:
                    for file_ in files:
                        buffer = io.BytesIO()
                        ftp.retrbinary('RETR ' + str(file_), buffer.write)
                        size = buffer.getbuffer().nbytes
                        print(f"Size of {file_} is: {size}")
                        tf = tarfile.TarInfo(file_)
                        tf.size = size
                        buffer.seek(0)
                        fout.addfile(tf,buffer)
                with tarfile.open(parquet_path, "r:*") as tar:
                    df = pd.concat([pd.read_parquet(tar.extractfile(file)) for file in tar.getnames()])
                    df.reset_index().to_feather(feather_path,compression = 'lz4')
                    yield pd.read_feather(feather_path)
    except error_perm as resp:
        if str(resp) == "550 No files found":
            print("No files in this directory")
        else:
            raise

In [5]:
targets_df,diseases_df,evidence_df = list(load_datasets_with_fallback())
display(targets_df)
display(diseases_df)
display(evidence_df)

,index,id,approvedSymbol,biotype,transcriptIds,genomicLocation,alternativeGenes,approvedName,go,hallmarks,...,obsoleteNames,constraint,tep,proteinIds,dbXrefs,chemicalProbes,homologues,tractability,safetyLiabilities,pathways
0,0,ENSG00000020219,CCT8L1P,processed_pseudogene,[ENST00000465400],"{'chromosome': '7', 'end': 152447150, 'start':...",None,"chaperonin containing TCP1 subunit 8 like 1, p...",None,None,...,"[{'label': 'chaperonin containing TCP1, subuni...",None,None,"[{'id': 'A6NM43', 'source': 'uniprot_obsolete'}]","[{'id': '32153', 'source': 'HGNC'}, {'id': 'IP...",None,None,None,None,None
1,1,ENSG00000059588,TARBP1,protein_coding,"[ENST00000496673, ENST00000484454, ENST0000048...","{'chromosome': '1', 'end': 234479179, 'start':...",None,TAR (HIV-1) RNA binding protein 1,"[{'aspect': 'F', 'ecoId': 'ECO:0007669', 'evid...",None,...,[{'label': 'Tar (HIV-1) RNA binding protein 1'...,"[{'constraintType': 'syn', 'exp': 272.51000976...",None,"[{'id': 'Q13395', 'source': 'uniprot_swissprot...","[{'id': '11568', 'source': 'HGNC'}, {'id': '2H...",None,"[{'homologyType': 'other_paralog', 'isHighConf...","[{'id': 'Approved Drug', 'modality': 'SM', 'va...",None,None
2,2,ENSG00000070182,SPTB,protein_coding,"[ENST00000553938, ENST00000389720, ENST0000055...","{'chromosome': '14', 'end': 64879907, 'start':...",None,"spectrin beta, erythrocytic","[{'aspect': 'P', 'ecoId': 'ECO:0007669', 'evid...",None,...,[],"[{'constraintType': 'syn', 'exp': 586.70001220...",None,"[{'id': 'P11277', 'source': 'uniprot_swissprot...","[{'id': '11274', 'source': 'HGNC'}, {'id': '1S...",None,"[{'homologyType': 'other_paralog', 'isHighConf...","[{'id': 'Approved Drug', 'modality': 'SM', 'va...",None,[{'pathway': 'COPI-mediated anterograde transp...
3,3,ENSG00000070366,SMG6,protein_coding,"[ENST00000354901, ENST00000570756, ENST0000057...","{'chromosome': '17', 'end': 2303785, 'start': ...",None,SMG6 nonsense mediated mRNA decay factor,"[{'aspect': 'P', 'ecoId': 'ECO:0000314', 'evid...",None,...,[{'label': 'chromosome 17 open reading frame 3...,"[{'constraintType': 'syn', 'exp': 321.14001464...",None,"[{'id': 'Q86US8', 'source': 'uniprot_swissprot...","[{'id': '17809', 'source': 'HGNC'}, {'id': '2D...",None,"[{'homologyType': 'ortholog_one2one', 'isHighC...","[{'id': 'Approved Drug', 'modality': 'SM', 'va...",None,[{'pathway': 'Nonsense Mediated Decay (NMD) en...
4,4,ENSG00000072071,ADGRL1,protein_coding,"[ENST00000361434, ENST00000589616, ENST0000059...","{'chromosome': '19', 'end': 14206187, 'start':...",[ENSG00000288324],adhesion G protein-coupled receptor L1,"[{'aspect': 'F', 'ecoId': 'ECO:0000250', 'evid...",None,...,"[{'label': 'latrophilin 1', 'source': 'HGNC'}]","[{'constraintType': 'syn', 'exp': 424.92999267...",None,"[{'id': 'O94910', 'source': 'uniprot_swissprot...","[{'id': '20973', 'source': 'HGNC'}, {'id': 'IP...",None,"[{'homologyType': 'other_paralog', 'isHighConf...","[{'id': 'Approved Drug', 'modality': 'SM', 'va...",None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60631,282,ENSG00000287783,ENSG00000287783,lncRNA,"[ENST00000667965, ENST00000662579]","{'chromosome': '5', 'end': 87774070, 'start': ...",None,novel transcript,None,None,...,[],None,None,None,[],None,None,None,None,None
60632,283,ENSG00000287887,ENSG00000287887,lncRNA,[ENST00000662637],"{'chromosome': '13', 'end': 24755201, 'start':...",None,novel transcript,None,None,...,[],None,None,None,[],None,None,None,None,None
60633,284,ENSG00000287908,ENSG00000287908,protein_coding,"[ENST00000477314, ENST00000474359]","{'chromosome': '12', 'end': 57626125, 'start':...",None,novel protein,None,None,...,[],None,None,"[{'id': 'E9PIH7', 'source': 'uniprot_trembl'}]",[],None,"[{'homologyType': 'within_species_paralog', 'i...",None,None,None
60634,285,ENSG00000287915,ENSG00000287915,lncRNA,[ENST00000662300],"{'chromosome': '15', 'end': 60223272, 'start':...",None,novel transcript,None,None,...,[],None,None,None,[],None,None,None,None,None


,index,id,code,dbXRefs,description,name,directLocationIds,obsoleteTerms,parents,sko,synonyms,ancestors,descendants,children,therapeuticAreas,indirectLocationIds,ontology
0,0,DOID_7551,http://purl.obolibrary.org/obo/DOID_7551,"[COHD:433417, ICD9:098.89, MeSH:D006069, NCIt:...",A primary bacterial infectious disease that is...,gonorrhea,None,None,"[EFO_0003955, MONDO_0000314]",None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[EFO_0000512, EFO_0009663, MONDO_0100336, OTAR...",[],[],"[MONDO_0100336, OTAR_0000017, EFO_0005741, EFO...",None,"{'isTherapeuticArea': False, 'leaf': True, 'so..."
1,1,HP_0000031,http://purl.obolibrary.org/obo/HP_0000031,"[UMLS:C0014534, SNOMEDCT_US:31070006, MSH:D004...",The presence of inflammation of the epididymis.,Epididymitis,None,None,"[HP_0012649, HP_0000022]",None,None,"[HP_0000022, HP_0000078, EFO_0000651, HP_00126...",[],[],[EFO_0000651],None,"{'isTherapeuticArea': False, 'leaf': True, 'so..."
2,2,HP_0000857,http://purl.obolibrary.org/obo/HP_0000857,[UMLS:C3278636],None,Neonatal insulin-dependent diabetes mellitus,None,None,"[EFO_0000400, HP_0001952, HP_0000818]",None,None,"[HP_0000818, HP_0000118, OTAR_0000020, HP_0011...",[],[],"[OTAR_0000020, EFO_0000651, EFO_0001379, EFO_0...",None,"{'isTherapeuticArea': False, 'leaf': True, 'so..."
3,3,HP_0001693,http://purl.obolibrary.org/obo/HP_0001693,"[UMLS:C0232180, SNOMEDCT_US:442119001]",Pattern of blood flow in the heart that deviat...,Cardiac shunt,None,None,[HP_0011028],None,None,"[EFO_0000651, HP_0011028, HP_0000118, HP_00110...",[HP_0012382],[HP_0012382],[EFO_0000651],None,"{'isTherapeuticArea': False, 'leaf': False, 's..."
4,4,HP_0001730,http://purl.obolibrary.org/obo/HP_0001730,[UMLS:C1842138],A progressive form of hearing impairment.,Progressive hearing impairment,None,None,[HP_0000364],None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[HP_0000364, EFO_0000651, HP_0000598, HP_0000118]",[HP_0000408],[HP_0000408],[EFO_0000651],None,"{'isTherapeuticArea': False, 'leaf': False, 's..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18701,243,Orphanet_93322,http://www.orpha.net/ORDO/Orphanet_93322,"[MESH:C535563, ICD10:Q72.5, MONDO:0010144, OMI...",Tibial hemimelia is a rare congenital anomaly ...,Tibial hemimelia,None,None,[Orphanet_2130],None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[EFO_0004260, Orphanet_404571, OTAR_0000006, O...","[Orphanet_295079, Orphanet_295077]","[Orphanet_295077, Orphanet_295079]","[OTAR_0000006, OTAR_0000018]",None,"{'isTherapeuticArea': False, 'leaf': False, 's..."
18702,244,Orphanet_93925,http://www.orpha.net/ORDO/Orphanet_93925,"[UMLS:C0431363, ICD10:Q04.2, MONDO:0019757, SC...",Alobar holoprosencephaly is the most severe cl...,Alobar holoprosencephaly,None,None,[Orphanet_2162],None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[Orphanet_183763, MONDO_0002025, MONDO_0019042...",[MONDO_0012322],[MONDO_0012322],"[MONDO_0002025, EFO_0000618, OTAR_0000018, EFO...",None,"{'isTherapeuticArea': False, 'leaf': False, 's..."
18703,245,Orphanet_98539,http://www.orpha.net/ORDO/Orphanet_98539,[],None,Early-onset ataxia with dementia,None,None,[Orphanet_98538],None,None,"[MONDO_0002025, Orphanet_183500, OTAR_0000018,...","[Orphanet_247815, EFO_0009016, Orphanet_284282...",[Orphanet_1172],"[MONDO_0002025, OTAR_0000018, EFO_0000618]",None,"{'isTherapeuticArea': False, 'leaf': False, 's..."
18704,246,Orphanet_98636,http://www.orpha.net/ORDO/Orphanet_98636,[MONDO:0020220],None,Corneoiridogoniodysgenesis,None,None,[Orphanet_98632],None,"{'hasBroadSynonym': None, 'hasExactSynonym': [...","[OTAR_0000018, Orphanet_183557, Orphanet_98632...","[Orphanet_98941, Orphanet_708, Orphanet_101033]",[Orphanet_708],"[OTAR_0000018, MONDO_0024458]",None,"{'isTherapeuticArea': False, 'leaf': False, 's..."


,index,datasourceId,targetId,alleleOrigins,allelicRequirements,beta,betaConfidenceIntervalLower,betaConfidenceIntervalUpper,biologicalModelAllelicComposition,biologicalModelGeneticBackground,...,targetModulation,textMiningSentences,urls,variantAminoacidDescriptions,variantFunctionalConsequenceId,variantId,variantRsId,diseaseId,id,score
0,0,eva,ENSG00000168646,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001819,17_65537821_C_T,rs1053887162,Orphanet_300576,000b404a55aab3d6e7df8d43501ec930a3f92fd8,0.02
1,1,eva,ENSG00000183873,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001583,3_38550706_T_C,rs1038605800,Orphanet_130,0010135b87920c6d0c95690fcc5380217479fe6b,0.32
2,2,eva,ENSG00000101605,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001589,18_3129232_CTCTG_C,None,EFO_0000538,002d610963d28704f2b207b4b14c47f2b55df14b,0.32
3,3,eva,ENSG00000169247,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001624,5_148998409_A_T,rs545169695,EFO_0004143,0030b182518da8abfc29d349fe478bfb66903132,0.02
4,4,eva,ENSG00000096696,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001574,6_7559224_A_G,None,Orphanet_247,003bb0a89a48ab62d4f57b2c0b6f1a29f099b4a3,0.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210868,5543,eva,ENSG00000168646,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001819,17_65558462_G_A,None,Orphanet_300576,ffdfa4b3c9a9212774a86a99f39beed20318d1a6,0.02
210869,5544,eva,ENSG00000169562,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001589,X_71224504_GC_G,rs1602350003,Orphanet_166,ffe4c57006027b9016c07f390c505e9bf8c5ab5a,0.90
210870,5545,eva,ENSG00000062822,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001819,19_50402359_C_T,rs746475325,Orphanet_140162,ffe88b1f2bbe8aaebff80111bde247fefbf090a3,0.02
210871,5546,eva,ENSG00000155657,[germline],None,NaN,NaN,NaN,None,None,...,None,None,None,None,SO_0001819,2_178729466_G_A,None,Orphanet_263,ffeaf543ce77a973514d395d2df49e94de8ec95e,0.32


In [6]:
evidence_df = evidence_df[['targetId','diseaseId','score']]
c = ['targetId', 'diseaseId']
# trgt_disease_with_median = evidence_df.groupby(c)['score'].median()
trgt_disease_with_medain_top3   = evidence_df.groupby(c,as_index=False).agg(median = pd.NamedAgg(column='score', aggfunc='median'),
                                                      top3   = pd.NamedAgg(column='score', aggfunc= lambda x: (x.nlargest(3).tolist())
                                                                              ))

In [7]:
joined_targets_df = trgt_disease_with_medain_top3.merge(targets_df[['id','approvedSymbol']],left_on='targetId',right_on='id')

joined_targets_df.drop(['id'],axis=1,inplace=True)
joined_targets_df

,targetId,diseaseId,median,top3,approvedSymbol
0,ENSG00000000419,HP_0001249,0.00,[0.0],DPM1
1,ENSG00000000419,Orphanet_137,0.32,"[0.92, 0.92, 0.9]",DPM1
2,ENSG00000000971,EFO_0001365,0.32,"[0.32, 0.32, 0.32]",CFH
3,ENSG00000000971,EFO_1001155,0.02,"[0.92, 0.32, 0.32]",CFH
4,ENSG00000000971,Orphanet_200421,0.02,"[0.02, 0.02, 0.02]",CFH
...,...,...,...,...,...
12057,ENSG00000288702,Orphanet_79235,0.32,[0.32],UGT1A3
12058,ENSG00000288705,EFO_0005556,0.32,"[0.9, 0.72, 0.5]",UGT1A5
12059,ENSG00000288705,Orphanet_2312,0.32,"[0.9, 0.32, 0.32]",UGT1A5
12060,ENSG00000288705,Orphanet_79234,0.32,"[0.9, 0.72, 0.32]",UGT1A5


In [8]:
joined_target_diesease_df = joined_targets_df.merge(diseases_df[['id','name']],left_on = 'diseaseId',right_on = 'id')
joined_target_diesease_df.drop(['id'],axis=1,inplace=True)
joined_target_diesease_df

,targetId,diseaseId,median,top3,approvedSymbol,name
0,ENSG00000000419,HP_0001249,0.00,[0.0],DPM1,Intellectual disability
1,ENSG00000004848,HP_0001249,0.00,[0.0],ARX,Intellectual disability
2,ENSG00000004961,HP_0001249,0.00,[0.0],HCCS,Intellectual disability
3,ENSG00000005302,HP_0001249,0.72,[0.72],MSL3,Intellectual disability
4,ENSG00000005339,HP_0001249,0.00,"[0.0, 0.0]",CREBBP,Intellectual disability
...,...,...,...,...,...,...
12057,ENSG00000284770,Orphanet_2333,0.72,[0.72],TBCE,Kenny-Caffey syndrome
12058,ENSG00000285053,MONDO_0014968,0.52,"[0.72, 0.32]",TBCE,"encephalopathy, progressive, with amyotrophy a..."
12059,ENSG00000285053,Orphanet_93324,0.72,[0.72],TBCE,Autosomal recessive Kenny-Caffey syndrome
12060,ENSG00000285708,EFO_0009908,0.72,[0.72],ENSG00000285708,glabellar hemangioma


In [9]:
joined_target_diesease_df.sort_values(by=['median']).to_json('output.json',orient= 'records')

print('Result for part one is now stored as output.json')

In [10]:
target_disease_df = joined_target_diesease_df[['targetId','diseaseId']]


In [11]:
result = pd.merge(target_disease_df,target_disease_df, on='diseaseId',suffixes =['','_right'])

result = result[result.targetId != result.targetId_right]

c = ['targetId','targetId_right']

target_target_disease_count = result.groupby(c,as_index=False).count()
target_target_disease_count = target_target_disease_count[target_target_disease_count.diseaseId >= 2]

print('Number of target-target pairs which share a connection to at least two diseases:',len(target_target_disease_count.index))

Number of target-target pairs which share a connection to at least two diseases: 87286
